In [ ]:
#plot(x=seq(1982,2017),y=aggregate(V_HC~V_VEH_YEAR, VTR2010, mean)[,2]/max(aggregate(V_HC~V_VEH_YEAR, VTR2010, mean)[,2]), xlab="Vehicle Year", ylab="HC/NOX/COavg (grams/mile)", main="HC/NOX/CO Emission by Vehicle Year", pch=15, cex.lab=1.4,col="blue")
#points(x=seq(1982,2017),aggregate(V_NOX~V_VEH_YEAR,VTR2010,mean)[,2]/max(aggregate(V_NOX~V_VEH_YEAR,VTR2010,mean)[,2]),pch=15, col="red")
#points(x=seq(1982,2017),aggregate(V_CO~V_VEH_YEAR,VTR2010,mean)[,2]/max(aggregate(V_CO~V_VEH_YEAR,VTR2010,mean)[,2]),pch=15, col="green")
#legend(2008,.7, c("HC","NOX","CO"),lty=c(1,1,1),lwd=c(2.5,2.5,2.5),col=c("blue","red","green"))

#the above lines of code create plots of emissions per model year for HC, NOX, CO. They are 
#normalized to 1 so they could be plotted on the same graph

VTR2010 <- read.csv("C:/Users/Allan/Downloads/VTR2010.txt")
#loads the relevant data



In [ ]:
VIN_cleaned <- trimws(VTR2010$V_VIN,"both")
VIN_cleaned <- as.data.frame(VIN_cleaned)
#makes a VIN category wihtout any white space beofre or after the VIN

VTR2010 <- transform(VTR2010, VIN_cleaned = VIN_cleaned)
#appends the cleaned vin category to the original dataset

VTR2010 <- VTR2010[nchar(as.character(VTR2010$VIN_cleaned)) == 17 ,]
#filters the data so only rows with a 17 digit cleaned_vin show up

new_vin_1 <- substr(VTR2010$VIN_cleaned, 2 ,4)
new_vin <-paste(new_vin_1, substr(VTR2010$VIN_cleaned, 8,8), sep = "")
new_vin <-paste(new_vin, substr(VTR2010$VIN_cleaned, 10,10), sep = "")
new_vin <- as.data.frame(new_vin)
#creates new field of truncated VIN numbers

VTR2010 <- cbind(VTR2010, new_vin)
#makes new data set with the new field in it


##COMPARE THESE TWO LINES
VTR2010 <- VTR2010[VTR2010$V_EM_RES == "P" | VTR2010$V_EM_RES == "F",]
nrow(VTR2010[VTR2010$V_EM_RES %in% c("P","F"),])


#filters out data so only P and F IM240 results are shown
VTR2010 <- subset(VTR2010, !is.na(V_HC & V_CO & V_NOX))
#VTR_VIN_NA_check <- VTR2010[!is.na(VTR2010$V_HC) & !is.na(VTR2010$V_CO) & !is.na(VTR2010$V_NOX),]
#all.equal(VTR2010, VTR_VIN_NA_check)
#filters out all NA values for HC, Co, NOX

VTR2010 <- VTR2010[VTR2010$V_HC > 0 & VTR2010$V_CO > 0 & VTR2010$V_NOX > 0 ,]
#gets rid of data that has all zeros for emissions results

sorted_list <- VTR2010[with(VTR2010, order(V_VIN, V_DATE_TIME, decreasing=FALSE)),] #no quotes around column names
#sort the data by date and vin

first_tests <- sorted_list[!duplicated(sorted_list$V_VIN, fromLast=FALSE),]
last_tests <- sorted_list[!duplicated(sorted_list$V_VIN, fromLast=TRUE),]
#get entries from unique vins for first tests only and last tests only

mini_VIN <- table(as.character(first_tests$new_vin))
mini_VIN_2010 <- as.data.frame(mini_VIN)
write.csv(mini_VIN_2010, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/mini_VIN_2010.csv")
#generates a table of all the unique minivins and their counts in the data set



In [ ]:
boxplot_2010 <- boxplot(mini_VIN_2010$Freq, xlab = "mini_vin_2010_count")
#generates a boxplot of the frequency of the minivins

summary_stats_2010 <- summary(mini_VIN_2010$Freq)
#does summary stats on the minivin frequency

write.csv(mini_VIN_2010, "mini_VIN_2010.csv")
#exports data

nrow(mini_VIN_2010[which(mini_VIN_2010$Freq <= 1),])

mini_vin_confirmation <- first_tests[,c("V_VEH_YEAR", "V_MAKE", "V_MODEL", "new_vin", "V_VIN", "V_TRANS", "V_CYLINDERS", "V_DISP", "V_DRIVE")]
#sorts cars out by certain VIN combo






In [ ]:
#create some objects
CO_data_list <- vector(mode="list", length=nrow(mini_VIN_2010)) #will hold summary stats for each minivin
Nox_data_list <- vector(mode="list", length=nrow(mini_VIN_2010)) #will hold summary stats for each minivin
HC_data_list <- vector(mode="list", length=nrow(mini_VIN_2010)) #will hold summary stats for each minivin

for (i in 1:nrow(mini_VIN_2010)) { #for each minivin
  if (i %% 100 ==0) {
    cat(paste(i, nrow(mini_VIN_2010), sep=": "), "\n")
  }
  mini_vin <- as.character(mini_VIN_2010$Var1)[i]
  #get the "i"th minivin
  
  subset_data <- first_tests[which(as.character(first_tests$new_vin) == mini_vin),]
  CO_data_list[[i]] <- summary(subset_data$V_CO)
  Nox_data_list[[i]] <- summary(subset_data$V_NOX)
  HC_data_list[[i]] <- summary(subset_data$V_HC)
}

CO_data_list <- do.call(rbind, CO_data_list)
Nox_data_list <- do.call(rbind, Nox_data_list)
HC_data_list <- do.call(rbind, HC_data_list)

mini_vin_summary_data <- cbind(mini_VIN_2010, CO_data_list, Nox_data_list, HC_data_list)
#genrates a table of summary stats of emissions for all the remaining mini vins in the data














In [ ]:
HC_Amount_first=aggregate(V_HC~V_VEH_YEAR, first_tests, mean)
NOX_amount_first=aggregate(V_NOX~V_VEH_YEAR,first_tests,mean)
CO_amount_first=aggregate(V_CO~V_VEH_YEAR,first_tests,mean)
#shows average emissions of first test by vehicle model year
Vehicle_year_first <- cbind(HC_Amount_first, NOX_amount_first, CO_amount_first)
write.csv(Vehicle_year_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Year_first_test_emissions_2010.csv")
#exports files as a csv file



In [ ]:
HC_Amount_Make_first=aggregate(V_HC~V_MAKE, first_tests, mean)
NOX_amount_Make_first=aggregate(V_NOX~V_MAKE,first_tests,mean)
CO_amount_Make_first=aggregate(V_CO~V_MAKE,first_tests,mean)
#shows average emissions of first test by vehicle make
Vehicle_make_first <- cbind(HC_Amount_Make_first, NOX_amount_Make_first, CO_amount_Make_first)
write.csv(Vehicle_make_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Make_first_test_emissions_2010.csv")
#exports files as a csv file

HC_Amount_Model_first=aggregate(V_HC~V_MODEL, first_tests, mean)
NOX_amount_Model_first=aggregate(V_NOX~V_MODEL,first_tests,mean)
CO_amount_Model_first=aggregate(V_CO~V_MODEL,first_tests,mean)
#shows average emissions of first test by model year
Vehicle_model_first <- cbind(HC_Amount_Model_first, NOX_amount_Model_first, CO_amount_Model_first)
write.csv(Vehicle_model_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Model_first_test_emissions_2010.csv")
#exports files as csv file

HC_Amount_VIN_first=aggregate(V_HC~new_vin, first_tests, mean)
NOX_amount_VIN_first=aggregate(V_NOX~new_vin,first_tests,mean)
CO_amount_VIN_first=aggregate(V_CO~new_vin,first_tests,mean)
#shows average emissions of first test by first 8 digits of the VIN
Vehicle_VIN_first <- cbind(HC_Amount_VIN_first, NOX_amount_VIN_first, CO_amount_VIN_first)
write.csv(Vehicle_VIN_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_VIN_first_test_emissions_2010.csv")
#exports files as csv file



In [ ]:
HC_Amount_last=aggregate(V_HC~V_VEH_YEAR, last_tests, mean)
NOX_amount_last=aggregate(V_NOX~V_VEH_YEAR,last_tests,mean)
CO_amount_last=aggregate(V_CO~V_VEH_YEAR,last_tests,mean)
#shows average emissions of last test by vehicle model year
Vehicle_year_last <- cbind(HC_Amount_last, NOX_amount_last, CO_amount_last)
write.csv(Vehicle_year_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_year_last_test_emissions_2010.csv")
#exports files as csv file

HC_Amount_Make_last=aggregate(V_HC~V_MAKE, last_tests, mean)
NOX_amount_Make_last=aggregate(V_NOX~V_MAKE,last_tests,mean)
CO_amount_Make_last=aggregate(V_CO~V_MAKE,last_tests,mean)
#shows average emissions of last test by vehicle make
Vehicle_Make_last <- cbind(HC_Amount_Make_last, NOX_amount_Make_last, CO_amount_Make_last)
write.csv(Vehicle_Make_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Make_last_test_emissions_2010.csv")
#exports files as csv file

HC_Amount_Model_last=aggregate(V_HC~V_MODEL, last_tests, mean)
NOX_amount_Model_last=aggregate(V_NOX~V_MODEL,last_tests,mean)
CO_amount_Model_last=aggregate(V_CO~V_MODEL,last_tests,mean)
#shows average emissions of last test by model year
Vehicle_Model_last <- cbind(HC_Amount_Model_last, NOX_amount_Model_last, CO_amount_Model_last)
write.csv(Vehicle_Model_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Model_last_test_emissions_2010.csv")
#exports files as a csv file

HC_Amount_VIN_last=aggregate(V_HC~new_vin, last_tests, mean)
NOX_amount_VIN_last=aggregate(V_NOX~new_vin,last_tests,mean)
CO_amount_VIN_last=aggregate(V_CO~new_vin,last_tests,mean)
#shows average emissions of last test by first 8 digits of the VIN
VIN_last <- cbind(HC_Amount_VIN_last, NOX_amount_VIN_last, CO_amount_VIN_last)
write.csv(VIN_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vin_last_2010.csv")
#exports files as csv file
